In [ ]:
import meshio
import numpy as np
from scipy.interpolate import RectBivariateSpline
import jigsawpy
import os
import matplotlib
import matplotlib.tri as mtri
import matplotlib.pyplot as plt
%matplotlib inline


from gLEC.gLEC import gLEC

In [ ]:
minx, maxx = -1e6, 1e6
miny, maxy = -1e6, 1e6
minz, maxz = 0., 4000.
dst_path = "outputs"


In [ ]:
def create_square_mesh(mesh_percentage_max_size):

    opts = jigsawpy.jigsaw_jig_t()

    geom = jigsawpy.jigsaw_msh_t()
    mesh = jigsawpy.jigsaw_msh_t()

    #------------------------------------ define JIGSAW geometry

    geom.mshID = "euclidean-mesh"
    geom.ndims = +2
    geom.vert2 = np.array([   # list of xy "node" coordinate
        ((minx, miny), 0),          # outer square
        ((maxx, miny), 0),
        ((maxx, maxy), 0),
        ((minx, maxy), 0),],
        dtype=geom.VERT2_t)

    geom.edge2 = np.array([   # list of "edges" between vert
        ((0, 1), 0),          # outer square
        ((1, 2), 0),
        ((2, 3), 0),
        ((3, 0), 0),],
        dtype=geom.EDGE2_t)

    #------------------------------------ build mesh via JIGSAW!

    opts.hfun_hmax = mesh_percentage_max_size

    opts.mesh_dims = +2                 # 2-dim. simplexes

    opts.optm_qlim = +.95

    opts.mesh_top1 = True               # for sharp feat's
    opts.geom_feat = True

    jigsawpy.lib.jigsaw(opts, geom, mesh)

    scr2 = jigsawpy.triscr2(            # "quality" metric
        mesh.point["coord"],
        mesh.tria3["index"])
    
    z = (mesh.point['coord'][:,0]/((maxx-minx)))**2 + (mesh.point['coord'][:,1]/((maxy-miny)))**2
    z = max(z) - z
    print(max(z), min(z))

    z[z<0.4] = 0.
    z = np.sin(z)
    z = ((z - np.min(z)) / (np.max(z) - np.min(z))) * (maxz - minz) + minz
    #z = np.ones_like(z)
    outmesh = meshio.Mesh(mesh.point['coord'], {'triangle': mesh.tria3['index']}, {'Z':z})
    
    return outmesh

In [ ]:
meshes = [create_square_mesh(0.06), create_square_mesh(0.04), create_square_mesh(0.02)]

fig, axes = plt.subplots(1, len(meshes), figsize=(15,5))

for i, mesh in enumerate(meshes):
    triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
    axes[i].triplot(triang)
    axes[i].set_aspect('equal')      

fig, axes = plt.subplots(1, len(meshes), figsize=(15,5))
for i, mesh in enumerate(meshes):
    triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
    _= axes[i].tripcolor(triang, mesh.point_data['Z'], edgecolors='none')    
    axes[i].set_aspect('equal')  

In [ ]:
glecs = [gLEC(mesh=mesh, max_fuel=2000) for mesh in meshes]

for g in glecs:
    print(g.normalised_area, g.max_fuel / g.horizontal_distance_cost_weight)

print((maxx-minx) * (maxy-miny))

In [ ]:
fig, axes = plt.subplots(1, len(meshes), figsize=(15,5))

for i, (mesh, glec) in enumerate(zip(meshes, glecs)):
    LEC_vals = []
    factor = 0.0

    for p in range(0, mesh.points.shape[0]):
        #LEC_vals.append((p, glec.get_normalised_area_covered_by_all_paths(p)))
        LEC_vals.append((p, glec.get_area_covered_by_all_paths(p)))

        # Print out the percentage complete, every 10% done.
        if p >= mesh.points.shape[0] * factor:
            print(f"{int(factor * 100):>3}% complete")
            factor += 0.1
    print("Done")

    mesh.point_data['LEC'] = np.zeros_like(mesh.point_data['Z'])
    for point, val in LEC_vals:
        mesh.point_data['LEC'][point] = val
        
    axes[i].set_aspect('equal')
    triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
    _= axes[i].tripcolor(triang, mesh.point_data['LEC'], edgecolors='none')
